In [1]:
import pandas as pd

## Query

In [2]:
from badassdatascience.forex.database.mongodb.pull_forex_data_from_database.pull_forex_data_from_database import pull_forex_into_df 
from badassdatascience.forex.database.mongodb.utilities.connect_to_mongoDB import get_candlestick_documents

In [3]:
instrument = 'EUR/USD'
granularity = 'M15'

pull_from_database = False
output_directory = 'TEMP'

In [4]:
import pickle

if pull_from_database:
    candlestick_documents = get_candlestick_documents()
    df = pull_forex_into_df(candlestick_documents, instrument = instrument, granularity = granularity)
    with open(output_directory + '/df.pickled', 'wb') as fff:
        pickle.dump(df, fff)
else:
    with open(output_directory + '/df.pickled', 'rb') as fff:
        df = pickle.load(fff)

In [5]:
df.head(2)

,_id,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
0,68097d0b1df60fc98f28d4fa,3,1262540700,EUR/USD,M15,2010-01-03T12:45:00-05:00,6,12,1.43070,1.43097,...,1.43247,0.00077,0.00077,1.43120,1.43172,1.43120,1.43172,0.00052,0.00052,2010-01-03 12:45:00-05:00
1,68097d0b1df60fc98f28d4fb,36,1262541600,EUR/USD,M15,2010-01-03T13:00:00-05:00,6,13,1.43097,1.43325,...,1.43185,-0.00062,0.00340,1.43172,1.43425,1.43105,1.43110,-0.00062,0.00320,2010-01-03 13:00:00-05:00


In [6]:
df.tail(2)

,_id,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
384263,6812f549e1775502615610d1,1157,1746072000,EUR/USD,M15,2025-05-01T00:00:00-04:00,3,0,1.13114,1.13137,...,1.13138,0.00008,0.00066,1.13122,1.13144,1.13078,1.13130,0.00008,0.00066,2025-05-01 00:00:00-04:00
384264,6812f8cc763c0ffbfd28dc9a,833,1746072900,EUR/USD,M15,2025-05-01T00:15:00-04:00,3,0,1.13121,1.13122,...,1.13102,-0.00035,0.00054,1.13129,1.13130,1.13076,1.13094,-0.00035,0.00054,2025-05-01 00:15:00-04:00


## Find all timestamps, stepped by granularity, for the full timestamp range

In [7]:
from badassdatascience.forex.utilities.granularities import get_df_merged_by_seconds_interval

In [8]:
df_to_ff = get_df_merged_by_seconds_interval(df)

In [9]:
df_to_ff.head(2)

,time,_id,volume,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
0,1262540700,68097d0b1df60fc98f28d4fa,3.0,EUR/USD,M15,2010-01-03T12:45:00-05:00,6.0,12.0,1.43070,1.43097,...,1.43247,0.00077,0.00077,1.43120,1.43172,1.43120,1.43172,0.00052,0.00052,2010-01-03 12:45:00-05:00
1,1262541600,68097d0b1df60fc98f28d4fb,36.0,EUR/USD,M15,2010-01-03T13:00:00-05:00,6.0,13.0,1.43097,1.43325,...,1.43185,-0.00062,0.00340,1.43172,1.43425,1.43105,1.43110,-0.00062,0.00320,2010-01-03 13:00:00-05:00


In [10]:
df_to_ff.tail(2)

,time,_id,volume,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
537257,1746072000,6812f549e1775502615610d1,1157.0,EUR/USD,M15,2025-05-01T00:00:00-04:00,3.0,0.0,1.13114,1.13137,...,1.13138,0.00008,0.00066,1.13122,1.13144,1.13078,1.13130,0.00008,0.00066,2025-05-01 00:00:00-04:00
537258,1746072900,6812f8cc763c0ffbfd28dc9a,833.0,EUR/USD,M15,2025-05-01T00:15:00-04:00,3.0,0.0,1.13121,1.13122,...,1.13102,-0.00035,0.00054,1.13129,1.13130,1.13076,1.13094,-0.00035,0.00054,2025-05-01 00:15:00-04:00


## Forward-fill

In [11]:
from badassdatascience.forex.utilities.forward_fill import ff

In [12]:
df_post_ff = ff(df_to_ff)

In [13]:
import datetime
from pytz import timezone
tz = timezone('US/Eastern')

# 16:00 rather than 17:00 because of daylight savings time
weekend_start_method_one = datetime.datetime(2025, 4, 25, 16, 0, 0, 0, tzinfo = tz).timestamp()
weekend_end_method_one = datetime.datetime(2025, 4, 27, 16, 0, 0, 0, tzinfo = tz).timestamp()

df_weekend_ff_test_one = df_post_ff[
    (df_post_ff['time'] >= weekend_start_method_one) & 
    (df_post_ff['time'] < weekend_end_method_one)
]

# check immediate surroundings of the previous interval
weekend_start_method_two = datetime.datetime(2025, 4, 25, 15, 0, 0, 0, tzinfo = tz).timestamp()
weekend_end_method_two = datetime.datetime(2025, 4, 27, 17, 0, 0, 0, tzinfo = tz).timestamp()

df_weekend_ff_test_two = df_post_ff[
    (df_post_ff['time'] >= weekend_start_method_two) & 
    (df_post_ff['time'] < weekend_end_method_two)
]

# final test
x_start = datetime.datetime(2025, 4, 24, 15, 0, 0, 0, tzinfo = tz).timestamp()
x_end = datetime.datetime(2025, 4, 30, 15, 0, 0, 0, tzinfo = tz).timestamp()
df_ff_final_test = df_post_ff[
    (df_post_ff['time'] >= x_start) & 
    (df_post_ff['time'] < x_end)
]

In [ ]:
import matplotlib.pyplot as plt

def rough_plot(x, y):
    plt.figure()
    plt.plot(x, y)
    plt.show()
    plt.close()

In [ ]:
rough_plot(df_weekend_ff_test_one['time'], df_weekend_ff_test_one['ask_c'])
rough_plot(df_weekend_ff_test_two['time'], df_weekend_ff_test_two['ask_c'])
rough_plot(df_ff_final_test['time'], df_ff_final_test['ask_c'])

## Plot against time

In [ ]:
def plot_time_series(
    df,
    time_column = 'time',
    dependent_column = 'mid_c',
    title = 'EUR/USD',
    granularity = 'M15',
    x_label = 'UNIX Epoch Time',
    y_label = 'Close (Mid)',
):
    plt.figure()
    plt.plot(df[time_column], df[dependent_column], label = title)
    plt.title(title + ' - ' + granularity)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.tight_layout()
    plt.show()
    plt.close()    

In [ ]:
plot_time_series(df_post_ff)

## Autocorrelations

In [ ]:
import numpy as np  # maybe?
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_autocorrelations_et_al(df, column_name = 'mid_c', lags = 500):
    x = df[column_name].values
    
    plt.figure()
    plot_acf(x, lags = lags)
    plt.tight_layout()
    plt.show()
    plt.close()

    plt.figure()
    plot_pacf(x, lags = lags)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
plot_autocorrelations_et_al(df_post_ff)

## Fourier

In [ ]:

### WRONG DF!

import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

def blah(df, column_name = 'mid_c'):

    #y = df[column_name].values[0:(len(df[column_name])):100]
    y = [df[column_name].values[1:] - df[column_name].values[0:-1]][0:(len(df[column_name])):100]
    
    #fft_signal = np.fft.fft(df[column_name].values)
    N = len(y)
    T = 1

    xf = fftfreq(N, T)[:N//2]
    yf = np.log10(fft(y))
    
    plt.figure()
    plt.plot(xf, yf[0:(len(xf))])
    plt.show()
    plt.close()

blah(df)

## pyTA

In [ ]:
from badassdatascience.forex.technical_analysis_indicators.technical_analysis_indicators import compute_ta_indicators
from badassdatascience.forex.utilities.quality_tests import does_df_have_only_one_instrument_and_granularity

In [ ]:
if does_df_have_only_one_instrument_and_granularity(df):
    df_tai = compute_ta_indicators(df, price_type_list = ['mid'])

In [ ]:
#df_tai = df_tai.dropna()

In [ ]:
df_tai

## Causality Investigation

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

In [ ]:
price_type = 'mid'

mean_hlc = (
    (
        (
            df[price_type + '_' + 'h'] +
            df[price_type + '_' + 'l'] +
            df[price_type + '_' + 'c']
        ) / 3.
    )
    .values
)

In [ ]:
mean_hlc

In [ ]:
test_column_list = [x for x in df_tai.columns if not x in ['time', 'instrument', 'granularity']]

In [ ]:
test_column_list[0:3]

In [ ]:
maxlag = 20
test = 'ssr_chi2test'

array_len = len(mean_hlc)
results_dict = {}
for column_name in test_column_list[0:3]:

    df_temp = pd.DataFrame({'mean_hlc' : mean_hlc, 'test_column' : df_tai[column_name].values})
    df_temp = df_temp.dropna()

    M = df_temp.to_numpy()
    test_result = grangercausalitytests(M, maxlag = maxlag, verbose = False)
    p_values = [test_result[i+1][0][test][1] for i in range(maxlag)]

    results_dict[column_name] = list(np.log10(p_values))

In [ ]:
import pprint as pp
pp.pprint(results_dict)

In [ ]:
# perform a self x self granger test as well...

## TA-Lib

In [14]:
import pandas as pd
import talib

In [15]:

def test_for_all_null(df, column_name):
    return df[column_name].isna().value_counts().get(True) == len(df[column_name])

def do_ta_calculations(df, column_name = 'mid_c', time_column_name = 'time'):
    x = df[column_name].values
    result_dict = {}
    result_len_list = []
    
    for item in dir(talib):
        if item.isupper() and item[0:2] != '__':

            try:
                # this gets the single-argument stuff
                result = eval('talib.' + item + '(x)')
            except:
                continue

            item_edited = item
            
            if str(type(result)) == "<class 'numpy.ndarray'>":
                result_dict[item_edited] = result
                result_len_list.append(len(result))
            
            elif str(type(result)) == "<class 'tuple'>":
                for i, array in enumerate(result):
                    item_edited = item + '_element_' + str(i)
                    result_dict[item_edited] = array
                    result_len_list.append(len(array))

            else:
                print()
                print('Something else found...')
                print(type(result))
                print()
                break
    
    df_new = pd.DataFrame(result_dict)

    column_list = df_new.columns
    columns_to_drop = []
    for col in column_list:
        if test_for_all_null(df_new, col):
            columns_to_drop.append(col)
    if len(columns_to_drop) > 0:
        df_new = df_new.drop(columns = columns_to_drop)

    df_new[time_column_name] = df[time_column_name]
    df_new[column_name] = df[column_name]
    df_new['instrument'] = df['instrument']
    df_new['granularity'] = df['granularity']

    final_column_list = [time_column_name, column_name, 'instrument', 'granularity']
    final_column_list.extend(column_list)

    return df_new[final_column_list], column_list

In [16]:
df_ta_results, ta_results_column_list = do_ta_calculations(df_post_ff)
df_ta_results
#ta_results_column_list

,time,mid_c,instrument,granularity,ACOS,APO,ASIN,ATAN,AVGDEV,BBANDS_element_0,...,SUM,T3,TAN,TANH,TEMA,TRIMA,TRIX,TSF,VAR,WMA
0,1262540700,1.43172,EUR/USD,M15,NaN,NaN,NaN,0.961104,NaN,NaN,...,NaN,NaN,7.143878,0.892019,NaN,NaN,NaN,NaN,NaN,NaN
1,1262541600,1.43110,EUR/USD,M15,NaN,NaN,NaN,0.960901,NaN,NaN,...,NaN,NaN,7.111758,0.891892,NaN,NaN,NaN,NaN,NaN,NaN
2,1262542500,1.43155,EUR/USD,M15,NaN,NaN,NaN,0.961049,NaN,NaN,...,NaN,NaN,7.135042,0.891984,NaN,NaN,NaN,NaN,NaN,NaN
3,1262543400,1.43106,EUR/USD,M15,NaN,NaN,NaN,0.960888,NaN,NaN,...,NaN,NaN,7.109696,0.891884,NaN,NaN,NaN,NaN,NaN,NaN
4,1262544300,1.43157,EUR/USD,M15,NaN,NaN,NaN,0.961055,NaN,1.431936,...,NaN,NaN,7.136081,0.891988,NaN,NaN,NaN,NaN,7.188000e-08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537254,1746069300,1.13054,EUR/USD,M15,NaN,-0.000731,NaN,0.846593,0.000595,1.131867,...,33.96207,1.130956,2.122720,0.811204,1.130689,1.132212,-0.007656,1.130379,1.589601e-07,1.131721
537255,1746070200,1.13016,EUR/USD,M15,NaN,-0.000797,NaN,0.846426,0.000551,1.131811,...,33.96005,1.130789,2.120629,0.811074,1.130501,1.132138,-0.007716,1.130225,2.435841e-07,1.131598
537256,1746071100,1.13122,EUR/USD,M15,NaN,-0.000806,NaN,0.846891,0.000421,1.131811,...,33.95883,1.130717,2.126469,0.811436,1.130535,1.132057,-0.007756,1.130473,2.435841e-07,1.131548
537257,1746072000,1.13130,EUR/USD,M15,NaN,-0.000757,NaN,0.846926,0.000352,1.131634,...,33.95768,1.130753,2.126911,0.811464,1.130586,1.131968,-0.007776,1.130695,1.840641e-07,1.131505


In [17]:
y_column_name = 'mid_c'
time_column_name = 'time'

y = df_ta_results[y_column_name].values
ts = df_ta_results[time_column_name].values

In [18]:
t = df_ta_results.transpose()
column_list = t.columns
t['combined']= t.values.tolist()
t.drop(columns = column_list, inplace = True)

y_list = []
for i in range(0, len(t.index)):
    y_list.append(y)

t[y_column_name] = y_list

t.reset_index(inplace = True)
t = t[t['index'].isin(ta_results_column_list)]

t

,index,combined,mid_c
4,ACOS,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
5,APO,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
6,ASIN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
7,ATAN,"[0.9611042689293725, 0.9609009192040567, 0.961...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
8,AVGDEV,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
...,...,...,...
75,TRIMA,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
76,TRIX,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
77,TSF,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."
78,VAR,"[nan, nan, nan, nan, 7.187999928603972e-08, 1....","[1.43172, 1.4311, 1.43155, 1.43106, 1.43157, 1..."


In [60]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

def gc_it(caused_by, causing, test = 'ssr_chi2test', lag = 20):
    df = pd.DataFrame({'caused_by' : caused_by, 'causing' : causing}).dropna()
    M = df.to_numpy()

    test_result = grangercausalitytests(M, maxlag = [lag], verbose = False)
    p_value = test_result[lag][0][test][1]

    return p_value

In [59]:
test = 'ssr_chi2test'
causing = t[t['index'] == 'APO']['combined'].values[0]
caused_by = list(t[t['index'] == 'APO'][y_column_name].values[0])
gc_it(caused_by, causing, test = test)

/home/emily/venvs/ml/lib/python3.10/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/emily/venvs/ml/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 50, but rank is 1
  warnings.warn('covariance of constraints does not have full '


1.0

In [ ]:
sdf = spark.createDataFrame(t)
sdf.show(2)